In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

2024-12-09 02:43:38.971569: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-09 02:43:38.973299: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-09 02:43:38.996954: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 02:43:38.996998: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 02:43:38.997835: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [36]:
### Load the trained model, scaler pickle,onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [37]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [38]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


/home/ice/code/project/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [39]:
input_df = pd.DataFrame([input_data])

In [40]:
## Encode categoriacal variable 

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [41]:
## concatenation with one hot encoded 
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)  

In [42]:
## SCaling the data 

input_scaled = scaler.transform(input_df)
input_scaled

/home/ice/code/project/venv/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


array([[ 6.00000000e+02,  1.00000000e+00,  4.00000000e+01,
         3.00000000e+00,  6.00000000e+04,  2.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  5.00000000e+04,
         1.00000000e+00,  7.10542736e-18, -4.26325641e-17]])

In [43]:
#prediction 
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 70ms/step


array([[1.]], dtype=float32)

In [44]:
prediction_proba=prediction[0][0]
prediction_proba

1.0

In [45]:
if prediction> 0.5:
    print("likely to churn")
else:
    print("not likely to churn")
    

likely to churn
